In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import logging
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection, naive_bayes, svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [2]:
data=pd.read_csv('Full+data.csv')
extra=pd.read_csv('extra_data.csv')
attributes=pd.read_excel('Womens+Attributes.xlsx')
USC_product_attributes=pd.read_excel('USC+Product+Attribute+Data+03302020.xlsx')
attributes=attributes[['Style',
       '(Reference Style Lookbook, choose all that apply)']]
tags=pd.read_csv('usc_additional_tags.csv')
#Dropped all columns with ALL null values
data_dropped=data.dropna(axis=1,how='all')
#Attach additional data to full data
full_data=pd.concat([data_dropped,extra])
full_data.drop_duplicates(subset=['product_id'], keep="first",inplace = True)

In [3]:
full_data=full_data[['product_id','description','details','name','brand_category']]

In [4]:
full_data.fillna('UNKNOWNTOKEN',inplace=True)

In [5]:
subset_for_model=full_data

In [6]:
# #See how many occasion exists, These are the occasions we need to look for and find rules for
# final_data=final_data[final_data['attribute_name']=='occasion']
# subset_for_model=final_data[['brand_category','description','details','name','product_id','attribute_value']]
#Remove / from brand_category by space
subset_for_model['brand_category']=subset_for_model['brand_category'].str.replace('/',' ')
subset_for_model['brand_category']=subset_for_model['brand_category'].str.replace(',',' ')
subset_for_model['details']=subset_for_model['brand_category'].str.replace('\n',' ')
subset_for_model['details']=subset_for_model['brand_category'].str.replace('/',' ')
subset_for_model['details']=subset_for_model['brand_category'].str.replace(',',' ')
subset_for_model.head()
subset_for_model.fillna('UNKNOWNTOKEN',inplace=True)


In [8]:
subset_for_model.head()

,product_id,description,details,name,brand_category
0,01DSRPSZTDW2PGK1YWYXJGKZZ0,Vintage Fitness leather sneakers with logo pri...,TheMensStore Shoes Sneakers LowTop,Original Fitness Sneakers,TheMensStore Shoes Sneakers LowTop
1,01DSQXJBX0R7DCW7KTAC1SW547,UNKNOWNTOKEN,Unknown,HAT,Unknown
2,01DPGV8TGRAB993PF7Z3YWG2VR,A Timeless Leather Belt Crafted From Smooth Co...,Accessories,Petit Oval Buckle Belt,Accessories
3,01DSR8G3F7DBRTMP8THF97XSQ2,Pretty ruffle sleeves and trim elevate essenti...,JustKids Girls214 Girls SwimwearCoverups JustK...,Little Gir's & Girl's Ariana One-Piece UPF 50+...,JustKids Girls214 Girls SwimwearCoverups JustK...
4,01DSR8G5GP519DEDCSKBMWQVK5,Versatile convertible gown with elephant applique,JustKids Baby024months InfantGirls FootiesRompers,Baby Girl's Endearing Elephants Pima Cotton Co...,JustKids Baby024months InfantGirls FootiesRompers


In [9]:
subset_for_model.isna().sum()

product_id        0
description       0
details           0
name              0
brand_category    0
dtype: int64

In [12]:
subset_for_model=subset_for_model.reset_index()

In [13]:
#Remove stopwords
nltk.download('stopwords')
columns=['details','description','name','brand_category']
for j in columns:
    corpus = []
    for i in range(0,len(subset_for_model)):
        new=re.sub('[^a-zA-Z]', ' ',str(subset_for_model.loc[i,j]))
        p=[]
        for k in new.split():
            separated=re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ',k)
            new = separated.lower()
            p.append(new)
        ls=WordNetLemmatizer()
        new = [ls.lemmatize(word) for word in p if not word in set(stopwords.words('english'))]
        new = ' '.join(new)
        subset_for_model.loc[i,j]=new

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/snehilsaraswat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
subset_for_model['details']=subset_for_model['details'].str.replace('dressesandjumpsuits','dresses and jumpsuits')
subset_for_model['brand_category']=subset_for_model['brand_category'].str.replace('dressesandjumpsuits','dresses and jumpsuits')

In [15]:
USC_product_attributes=pd.read_excel('USC+Product+Attribute+Data+03302020.xlsx')
new_tags=pd.concat([tags,USC_product_attributes])

In [16]:
new_tags=new_tags[['product_id','attribute_name','attribute_value']]

In [17]:
new_occ_tags=new_tags[new_tags['attribute_name']=='occasion']

In [18]:
new_occ_tags=new_occ_tags.reset_index()

In [19]:
new_occ_tags['attribute_value']=new_occ_tags['attribute_value'].str.lower()
new_occ_tags['attribute_value']=new_occ_tags['attribute_value'].str.replace(' ','')

In [20]:
new_occ_tags['attribute_value'].unique()

array(['weekend', 'workout', 'daytonight', 'coldweather', 'vacation',
       'nightout', 'work'], dtype=object)

In [21]:
#as we can see there can be multiple value so we create flag for each type of product_tag
list_of_occasions = new_occ_tags['attribute_value'].unique()
for j in list_of_occasions:
    for i in range(0,len(new_occ_tags)):
        if(new_occ_tags.loc[i,'attribute_value'] == j):
            new_occ_tags.loc[i,'is_'+j] = 1
        else:
            new_occ_tags.loc[i,'is_'+j] = 0

In [22]:
new_occ_tags.head()

,index,product_id,attribute_name,attribute_value,is_weekend,is_workout,is_daytonight,is_coldweather,is_vacation,is_nightout,is_work
0,31579,01E2P1MQS0GQ4SE93SY68ZYMCC,occasion,weekend,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,31580,01E2P1MQS0GQ4SE93SY68ZYMCC,occasion,weekend,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,31605,01E2P2QPCCQAN8WCHQKZY7EG73,occasion,workout,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,31611,01E2P2Q2WXXD75B1F4JCAKC99D,occasion,daytonight,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,31612,01E2P2Q2WXXD75B1F4JCAKC99D,occasion,daytonight,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [23]:
work=new_occ_tags[['product_id','is_work']]
tagged_work=work.groupby(['product_id'])['is_work'].max().reset_index()

nightout=new_occ_tags[['product_id','is_nightout']]
tagged_nightout=nightout.groupby(['product_id'])['is_nightout'].max().reset_index()

daytonight=new_occ_tags[['product_id','is_daytonight']]
tagged_daytonight=daytonight.groupby(['product_id'])['is_daytonight'].max().reset_index()

weekend=new_occ_tags[['product_id','is_weekend']]
tagged_weekend=weekend.groupby(['product_id'])['is_weekend'].max().reset_index()

vacation=new_occ_tags[['product_id','is_vacation']]
tagged_vacation=vacation.groupby(['product_id'])['is_vacation'].max().reset_index()

workout=new_occ_tags[['product_id','is_workout']]
tagged_workout=workout.groupby(['product_id'])['is_workout'].max().reset_index()

coldweather=new_occ_tags[['product_id','is_coldweather']]
tagged_coldweather=coldweather.groupby(['product_id'])['is_coldweather'].max().reset_index()

In [24]:
### Now, we need to add different tags one final tag df
df_list = [tagged_nightout,tagged_daytonight,tagged_vacation,tagged_workout,tagged_weekend,tagged_coldweather]
for df_ in df_list:
    tagged_work = pd.merge(tagged_work, df_, on='product_id', how='left')
tags_style_all = tagged_work

In [25]:
modeling=pd.merge(subset_for_model,tags_style_all,on='product_id', how = 'inner')

In [27]:
#Save to CSV
modeling.to_csv('modeling.csv')

In [15]:
tags=['is_work', 'is_nightout', 'is_daytonight', 'is_vacation', 'is_workout',
       'is_weekend', 'is_coldweather']

# TFIDF Vectorizer


In [121]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
vectorizer = TfidfVectorizer(min_df=10)

In [122]:
columns=['brand_category','details','description','name']
e=pd.DataFrame()
for i in columns:
    corpus = list(modeling[i].values)
    X = vectorizer.fit_transform(corpus)
    terms = vectorizer.get_feature_names()
    print(len(terms))
    c=pd.DataFrame(X.toarray().transpose(), index=terms)
    e=pd.concat([e,c])

49
49
1033
237


In [123]:
X=e.T.values
y=modeling['is_work'].values
# scaler = StandardScaler()
# X_std = scaler.fit_transform(X)

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [125]:
logreg=LogisticRegression(n_jobs=1, C=0.01)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_test,y_pred))

accuracy 0.7164750957854407


In [126]:
modeling['is_work'].sum()/len(modeling)

0.3428717424629535

In [127]:
tags=['is_work', 'is_nightout', 'is_daytonight', 'is_vacation', 'is_workout',
       'is_weekend', 'is_coldweather']

# Logistic One vs Rest

In [128]:
X=e.T.values
for i in tags:
    y=modeling[i].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    logreg=LogisticRegression(n_jobs=1, C=0.01)
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    print('accuracy %s' % accuracy_score(y_test,y_pred),"for tag",i)
    print("% for ", i, "is",1-(modeling[i].sum()/len(modeling)))

accuracy 0.7164750957854407 for tag is_work
% for  is_work is 0.6571282575370465
accuracy 0.7662835249042146 for tag is_nightout
% for  is_nightout is 0.7669902912621359
accuracy 0.722860791826309 for tag is_daytonight
% for  is_daytonight is 0.29790495656617266
accuracy 0.8518518518518519 for tag is_vacation
% for  is_vacation is 0.8346959632089934
accuracy 0.9553001277139208 for tag is_workout
% for  is_workout is 0.9601430761369443
accuracy 0.7816091954022989 for tag is_weekend
% for  is_weekend is 0.2350536535513541
accuracy 0.9246487867177522 for tag is_coldweather
% for  is_coldweather is 0.9348492590700052


In [246]:
sum(y_pred)

6

# Neural Network

In [129]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

for i in tags:
    y=modeling[i].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    size=[(128,64),(100,50),(50,20)]
    for j in size:
            nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=j).fit(X_train, y_train)
            y_pred_RF = nn.predict(X_test)
            #print("Actual % of the data for tag",p,"is",sum(subset_for_model['attribute_value'])/len(subset_for_model)*100)
            print(accuracy_score(y_test,y_pred_RF),j,i)
            print("% for ", i, "is",1-(modeling[i].sum()/len(modeling)),(modeling[i].sum()/len(modeling)),'\n')
    

0.7803320561941252 (128, 64) is_work
% for  is_work is 0.6571282575370465 0.3428717424629535 

0.7650063856960408 (100, 50) is_work
% for  is_work is 0.6571282575370465 0.3428717424629535 

0.789272030651341 (50, 20) is_work
% for  is_work is 0.6571282575370465 0.3428717424629535 

0.768837803320562 (128, 64) is_nightout
% for  is_nightout is 0.7669902912621359 0.23300970873786409 

0.768837803320562 (100, 50) is_nightout
% for  is_nightout is 0.7669902912621359 0.23300970873786409 

0.7650063856960408 (50, 20) is_nightout
% for  is_nightout is 0.7669902912621359 0.23300970873786409 

0.7215836526181354 (128, 64) is_daytonight
% for  is_daytonight is 0.29790495656617266 0.7020950434338273 

0.7139208173690932 (100, 50) is_daytonight
% for  is_daytonight is 0.29790495656617266 0.7020950434338273 

0.7049808429118773 (50, 20) is_daytonight
% for  is_daytonight is 0.29790495656617266 0.7020950434338273 

0.842911877394636 (128, 64) is_vacation
% for  is_vacation is 0.8346959632089934 0.16

In [156]:
sum(np.where(subset_for_model['attribute_value']=='work',1,0))

2410

# Naive Bayes

In [234]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [235]:
y_pred_NB = classifier.predict(X_test)

In [236]:
accuracy_score(y_test,y_pred_NB)

0.1473414477898783

# SVM Linear

In [130]:
for i in tags:
    y=modeling[i].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    SVM = svm.SVC(C=0.1, kernel='linear', degree=3, gamma='auto')
    SVM.fit(X_train,y_train)
    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(X_test)
    # Use accuracy_score function to get the accuracy
    print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)
    print("Random % for ", i, "is",max((1-(modeling[i].sum()/len(modeling))),(modeling[i].sum()/len(modeling))),'\n')

SVM Accuracy Score ->  78.67177522349937
Random % for  is_work is 0.6571282575370465 

SVM Accuracy Score ->  77.2669220945083
Random % for  is_nightout is 0.7669902912621359 

SVM Accuracy Score ->  74.07407407407408
Random % for  is_daytonight is 0.7020950434338273 

SVM Accuracy Score ->  86.71775223499361
Random % for  is_vacation is 0.8346959632089934 

SVM Accuracy Score ->  95.40229885057471
Random % for  is_workout is 0.9601430761369443 

SVM Accuracy Score ->  79.05491698595148
Random % for  is_weekend is 0.7649463464486459 

SVM Accuracy Score ->  92.33716475095785
Random % for  is_coldweather is 0.9348492590700052 



# Word 2 Vec

In [246]:
import gensim
from gensim.models import Word2Vec
wv = gensim.models.KeyedVectors.load_word2vec_format("desktop/nlp/project/GoogleNews-vectors-negative300.bin", binary=True)
wv.init_sims(replace=True)

In [244]:
from itertools import islice
list(islice(wv.vocab, 13030, 13050))

NameError: name 'wv' is not defined

In [ ]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [ ]:
X=modeling[['description','details','name','brand_category']]

In [ ]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
names=['description','name','details','brand_category']
c=pd.DataFrame()
for i in names:
    X_new = X.apply(lambda r: w2v_tokenize_text(r[i]),axis=1).values
    X_word_average = word_averaging_list(wv,X_new)
    X_array=pd.DataFrame(X_word_average)
    c=pd.concat([c,X_array],axis=1)

In [44]:
c.shape

(3914, 1200)

In [152]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

for i in tags:
    y=modeling[i].values
    X_train, X_test, y_train, y_test = train_test_split(c.values, y, test_size = 0.2, random_state = 0)
    size=[(128,64),(100,50),(50,20)]
    for j in size:
        nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=j).fit(X_train, y_train)
        y_pred_RF = nn.predict(X_test)
        #print("Actual % of the data for tag",p,"is",sum(subset_for_model['attribute_value'])/len(subset_for_model)*100)
        print(accuracy_score(y_test,y_pred_RF),j,i)
        print("% for ", i, "is",max((1-(modeling[i].sum()/len(modeling))),(modeling[i].sum()/len(modeling))),'\n')

0.7547892720306514 (128, 64) is_work
% for  is_work is 0.6571282575370465 

0.7484035759897829 (100, 50) is_work
% for  is_work is 0.6571282575370465 

0.7650063856960408 (50, 20) is_work
% for  is_work is 0.6571282575370465 

0.7420178799489144 (128, 64) is_nightout
% for  is_nightout is 0.7669902912621359 

0.7547892720306514 (100, 50) is_nightout
% for  is_nightout is 0.7669902912621359 

0.7381864623243933 (50, 20) is_nightout
% for  is_nightout is 0.7669902912621359 

0.7151979565772669 (128, 64) is_daytonight
% for  is_daytonight is 0.7020950434338273 

0.7203065134099617 (100, 50) is_daytonight
% for  is_daytonight is 0.7020950434338273 

0.7241379310344828 (50, 20) is_daytonight
% for  is_daytonight is 0.7020950434338273 

0.8518518518518519 (128, 64) is_vacation
% for  is_vacation is 0.8346959632089934 

0.8620689655172413 (100, 50) is_vacation
% for  is_vacation is 0.8346959632089934 

0.8403575989782887 (50, 20) is_vacation
% for  is_vacation is 0.8346959632089934 

0.957854

In [153]:
for i in tags:
    y=modeling[i].values
    X_train, X_test, y_train, y_test = train_test_split(c.values, y, test_size = 0.2, random_state = 0)
    SVM = svm.SVC(C=0.1, kernel='linear', degree=3, gamma='auto')
    SVM.fit(X_train,y_train)
    # predict the labels on validation dataset
    predictions_SVM = SVM.predict(X_test)
    # Use accuracy_score function to get the accuracy
    print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)
    print("Random % for ", i, "is",max((1-(modeling[i].sum()/len(modeling))),(modeling[i].sum()/len(modeling))),'\n')

SVM Accuracy Score ->  75.86206896551724
Random % for  is_work is 0.6571282575370465 

SVM Accuracy Score ->  76.62835249042146
Random % for  is_nightout is 0.7669902912621359 

SVM Accuracy Score ->  73.5632183908046
Random % for  is_daytonight is 0.7020950434338273 

SVM Accuracy Score ->  85.18518518518519
Random % for  is_vacation is 0.8346959632089934 

SVM Accuracy Score ->  95.53001277139208
Random % for  is_workout is 0.9601430761369443 

SVM Accuracy Score ->  78.16091954022988
Random % for  is_weekend is 0.7649463464486459 

SVM Accuracy Score ->  92.46487867177522
Random % for  is_coldweather is 0.9348492590700052 



In [ ]:
#Source of the code logic - https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568

# Deep learning Keras with Word2Vec

In [34]:
from keras.models import Sequential
from keras import layers

Using TensorFlow backend.


In [35]:
from sklearn.manifold import TSNE

In [ ]:
#Feed Word2Vec vectors in KERAS

In [58]:
for i in tags:
    y=modeling[i].values
    X_train, X_test, y_train, y_test = train_test_split(c.values, y, test_size = 0.2, random_state = 0)
    input_dim = X_train.shape[1]  # Number of features
    model = Sequential()
    model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=20,verbose=False,validation_data=(X_test, y_test),batch_size=10)
    loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
    print("Training Accuracy: {:.4f}".format(accuracy),"for",i)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
    print("Testing Accuracy:  {:.4f}".format(accuracy),"for",i)

Training Accuracy: 0.8400 for is_work
Testing Accuracy:  0.8008 for is_work
Training Accuracy: 0.8569 for is_nightout
Testing Accuracy:  0.7765 for is_nightout
Training Accuracy: 0.8330 for is_daytonight
Testing Accuracy:  0.7676 for is_daytonight
Training Accuracy: 0.8952 for is_vacation
Testing Accuracy:  0.8863 for is_vacation
Training Accuracy: 0.9939 for is_workout
Testing Accuracy:  0.9681 for is_workout
Training Accuracy: 0.8323 for is_weekend
Testing Accuracy:  0.8008 for is_weekend
Training Accuracy: 0.9559 for is_coldweather
Testing Accuracy:  0.9361 for is_coldweather


In [48]:
#This simple Keras Sequential Model with word2Vec does better than Rest of the models for us

# Keras with our own embeddings

In [4]:
modeling=pd.read_csv('modeling.csv')

In [217]:
modeling['all_data']=modeling['details']+modeling['name']+modeling['description']+modeling['brand_category']

In [206]:
#Keras Tokenizer
from keras.preprocessing.text import Tokenizer
columns=['brand_category','details','description','name']
for i in ['brand_category']:
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(modeling[i])
    X = tokenizer.texts_to_sequences(modeling[i])
    # Adding 1 because of reserved 0 index
    CD=tokenizer.word_index
    maxlen = 100
    X = pad_sequences(X, padding='post', maxlen=maxlen)
for i in ['details','description','name']:
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(modeling[i])
    p = tokenizer.texts_to_sequences(modeling[i])# Adding 1 because of reserved 0 index
    CD={**CD,**tokenizer.word_index}
    vocab_size=len(CD)
    p = pad_sequences(p, padding='post', maxlen=maxlen)
    X=np.concatenate((X,p),axis=1)   

In [221]:
#Keras Tokenizer
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(modeling['all_data'])
X = tokenizer.texts_to_sequences(modeling['all_data'])
# Adding 1 because of reserved 0 index
vocab_size=len(tokenizer.word_index)+1
maxlen = 1000
X = pad_sequences(X, padding='post', maxlen=maxlen)
# for i in ['details','description','name']:
#     tokenizer = Tokenizer()
#     tokenizer.fit_on_texts(modeling[i])
#     p = tokenizer.texts_to_sequences(modeling[i])# Adding 1 because of reserved 0 index
#     CD={**CD,**tokenizer.word_index}
#     vocab_size=len(CD)
#     p = pad_sequences(p, padding='post', maxlen=maxlen)
#     X=np.concatenate((X,p),axis=1)   

In [225]:
vocab_size

11152

In [224]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50
model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_154"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 1000, 50)          557600    
_________________________________________________________________
global_max_pooling1d_18 (Glo (None, 50)                0         
_________________________________________________________________
dense_305 (Dense)            (None, 10)                510       
_________________________________________________________________
dense_306 (Dense)            (None, 1)                 11        
Total params: 558,121
Trainable params: 558,121
Non-trainable params: 0
_________________________________________________________________


In [226]:
for i in tags:
    y=modeling[i].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    input_dim = X_train.shape[1]  # Number of features
    model = Sequential()
    model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=10,verbose=False,validation_data=(X_test, y_test),batch_size=5)
    loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
    print("Training Accuracy: {:.4f}".format(accuracy),"for",i)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
    print("Testing Accuracy:  {:.4f}".format(accuracy),"for",i)

Training Accuracy: 0.6509 for is_work
Testing Accuracy:  0.6897 for is_work
Training Accuracy: 0.7681 for is_nightout
Testing Accuracy:  0.7637 for is_nightout
Training Accuracy: 0.7055 for is_daytonight
Testing Accuracy:  0.6922 for is_daytonight
Training Accuracy: 0.8333 for is_vacation
Testing Accuracy:  0.8506 for is_vacation
Training Accuracy: 0.9614 for is_workout
Testing Accuracy:  0.9540 for is_workout
Training Accuracy: 0.7566 for is_weekend
Testing Accuracy:  0.7791 for is_weekend
Training Accuracy: 0.9384 for is_coldweather
Testing Accuracy:  0.9234 for is_coldweather


# Keras with Pre-trained Glove Embeddings

In [236]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [238]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix('desktop/nlp/project/glove.6B.300d.txt',tokenizer.word_index, embedding_dim)

In [239]:
#34% of vocabulary is covered through pre trained embeddings
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.4425215208034433

In [240]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_163"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 1000, 50)          557600    
_________________________________________________________________
global_max_pooling1d_19 (Glo (None, 50)                0         
_________________________________________________________________
dense_321 (Dense)            (None, 10)                510       
_________________________________________________________________
dense_322 (Dense)            (None, 1)                 11        
Total params: 558,121
Trainable params: 521
Non-trainable params: 557,600
_________________________________________________________________


In [243]:
for i in tags:
    y=modeling[i].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
    input_dim = X_train.shape[1]  # Number of features
    model = Sequential()
    model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=20,verbose=False,validation_data=(X_test, y_test),batch_size=10)
    loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
    print("Training Accuracy: {:.4f}".format(accuracy),"for",i)
    loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
    print("Testing Accuracy:  {:.4f}".format(accuracy),"for",i)

Training Accuracy: 0.6573 for is_work
Testing Accuracy:  0.6909 for is_work
Training Accuracy: 0.7697 for is_nightout
Testing Accuracy:  0.7612 for is_nightout
Training Accuracy: 0.7189 for is_daytonight
Testing Accuracy:  0.6833 for is_daytonight
Training Accuracy: 0.8333 for is_vacation
Testing Accuracy:  0.8480 for is_vacation
Training Accuracy: 0.9617 for is_workout
Testing Accuracy:  0.9553 for is_workout
Training Accuracy: 0.7643 for is_weekend
Testing Accuracy:  0.7752 for is_weekend
Training Accuracy: 0.9371 for is_coldweather
Testing Accuracy:  0.9132 for is_coldweather


In [ ]:
#Problems for low accuracy
1. There are not enough training samples
2. The data you have does not generalize well
3. Missing focus on tweaking the hyperparameters